In [109]:
from transformers import AutoModelForSequenceClassification, BertTokenizer
import torch
import pandas as pd
from tqdm import tqdm
from peft import PeftModel

In [110]:
# 加载模型和 tokenizer
# model = AutoModelForSequenceClassification.from_pretrained("./test_trainer-sft/checkpoint-11250")
model = AutoModelForSequenceClassification.from_pretrained("./test_trainer-sft-mix/checkpoint-60000")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# 读取测试数据
test_df = pd.read_csv("./data/test/NSL-KDD-10000.csv")
# test_df = pd.read_csv("./data/test/KDD99-10000.csv")
# test_df = pd.read_csv("./data/test/UNSW_NB15-10000.csv")
# test_df = pd.read_csv("./data/test/X-IIoTID-10000.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Gamma.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Gaussian.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Laplace.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Poisson.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Uniform.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Cauchy.csv")
texts = test_df["flow"].tolist()

In [111]:
# 遍历数据，找到最长文本的长度
text_lengths = [len(tokenizer(text)["input_ids"]) for text in texts]
max_length = max(text_lengths)
max_length = min(max(text_lengths), 500)
max_length

123

In [112]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

In [113]:
# 推理配置
model.eval()
torch.set_grad_enabled(False)
device = torch.device("cuda")
model.to(device)
# 进度条 & 批量推理
batch_size = 16  # 设置合适的 batch size
predictions = []
for i in tqdm(range(0, len(texts), batch_size), desc="预测进度", unit="batch"):
    batch_texts = texts[i : i + batch_size]
    inputs = tokenize_function(batch_texts)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    # 运行模型预测
    outputs = model(**inputs)
    batch_predictions = torch.argmax(outputs.logits, dim=-1).tolist()
    predictions.extend(batch_predictions)

预测进度: 100%|██████████| 625/625 [00:25<00:00, 24.83batch/s]


In [114]:
# 计算指标
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
if "class" in test_df.columns:
    true_labels = test_df["class"].tolist()
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')  # 二分类使用binary
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')
    print(f"\n模型在测试集上的指标:")
    print(f"准确率 (Accuracy): {accuracy:.4f}")
    print(f"精确率 (Precision): {precision:.4f}")
    print(f"召回率 (Recall): {recall:.4f}")
    print(f"F1 分数 (F1-score): {f1:.4f}")


模型在测试集上的指标:
准确率 (Accuracy): 0.9991
精确率 (Precision): 0.9991
召回率 (Recall): 0.9993
F1 分数 (F1-score): 0.9992
